In [1]:
import math
import numpy as np

In [52]:
def getfocus(a,ecc):
    c=a*ecc
    return c

In [2]:
import math
def InitialDataEqualMassConic(radius,ecc,angle,initmass):
    orbitalangle=angle
    print("angle",angle)
    phi=np.array([math.pi+orbitalangle,orbitalangle])
    orbitalradius=radius #semimajor axis
    eccentricity=ecc
    print(eccentricity)
    mass=np.ones(2)
    masses=initmass*mass #*masssun (natural units)
    return phi,orbitalangle,orbitalradius,eccentricity, masses
    

In [3]:
import random,numpy as np
rad0=50
ecc0=0.5
theta0=0. #math.pi/6.
mass0=1.0
initdateqellipse=InitialDataEqualMassConic(rad0,ecc0,theta0,mass0)
print(initdateqellipse)

angle 0.0
0.5
(array([3.14159265, 0.        ]), 0.0, 50, 0.5, array([1., 1.]))


In [14]:
def getxyuveqtwoellipses(initdat):
    phi,orbitangle,orbitalradius,eccentricity, masses=initdat
    print("initdat", orbitalradius, phi, eccentricity, np.cos(phi), np.sin(phi))
    metersperAU=1
    Gconstant=1
    #fix x0 y0 at one star, disregard initial data, use orbital radius as separation between stars
    #this is consistant with choice in previous part
    cosphi=np.cos(phi)
    sinphi=np.sin(phi)
    coordsep=orbitalradius #/2.
    print("coordsep",coordsep)
    #x0=orbitalradius/2.*np.cos(phi)*metersperAU
    #y0=orbitalradius/2.*np.sin(phi)*metersperAU
    count=0
    for phi0 in phi:
        if phi0==0:
            print("zero")
            #x0[count]=orbitalradius[count]/2.
            #y0[count]=0
            cosphi[count]=1.0
            sinphi[count]=0.0
        if phi0==math.pi:
            cosphi[count]=-1.0
            sinphi[count]=0.0
            print("pi")
            #x0[count]=-orbitalradius[count]/2.
            #y0[count]=0
        if phi0==math.pi/2.:
            cosphi[count]=0.0
            sinphi[count]=1.0
            #x0[count]=0
            #y0[count]=orbitalradius[count]/2.
        if phi0==3*math.pi/2.:
            cosphi[count]=0.0
            sinphi[count]=-1.0
            #x0[count]=0
            #y0[count]=-orbitalradius[count]/2.
        count+=1
    x0=coordsep*cosphi
    y0=coordsep*sinphi
    #x0[1]=0.0
    #y0[1]=0.0
    z0=np.zeros(2)
    print(x0)
    print(y0)
    v=np.zeros(2)
    a=np.zeros(2)
    ux0=0.
    uy0=0.
    uz0=0.
    ax0=0.
    ay0=0.
    az0=0.
    vx0=0.
    vy0=0.
    vz0=0.
    print(eccentricity)
    #masses=np.zeros(2)
    #masses[0]=masses[1]+masses[0]
    #masses[1]=(masses[1]*masses[0])/(masses[1]+masses[0])
    if eccentricity<1 and eccentricity>0:
        #elliptical
        orbitalr=orbitalradius #NO REDUCED MASS 
        #focusdisp=getfocus(orbitalr,eccentricity) #displacement of the center due to the focus being at the center
        coordsep=orbitalr*(1+eccentricity) #start at aphelion
        x0=(coordsep)*cosphi
        y0=(coordsep)*sinphi
        starsep=np.sqrt((x0[0]-x0[1])**2+(y0[0]-y0[1])**2) #two stars, at opposite ends of the orbit
        Fapastron=masses[1]*masses[0]/starsep**2
        #print('xycomp',coordsep,focusdisp,x0,y0)
        #x0[0]=0.0
        #y0[0]=0.0
        v=np.zeros(2)
        #v= np.sqrt(masses[1]*masses[0]/masses*(2./coordsep-1./(orbitalr)))
        v= np.sqrt(masses[1]*masses[0]/masses*(2./starsep-1./(2.*orbitalr)))
        ux0=-v*sinphi
        uy0=v*cosphi #initial data in y only 
        uz0=np.zeros(2)
    
        print("vstuff", coordsep,orbitalr, 2./coordsep, 1./orbitalr, (2./starsep-1./(2.*orbitalr)), 1./(coordsep))
        
        a=np.zeros(2)
        a=Fapastron/masses
    
        ax0=-a*cosphi
        ay0=-a*sinphi
        az0=np.zeros(2)
    elif eccentricity[0]==0.0: #circular
        #start at perihelion for both (eliptical, doesn't generalize to three body)
        #actually start with circular orbit
        ux0=np.zeros(2) #*149597870700
        #centrepital force balances gravitational force
        metersperAU=1 #natural units
        #G=1
        Gconstant=1
        #Fcentripital=mass1*v**2/rphys
        #centF=accel
        #Faccel=G*m1*m2/r^2
        r0=2.*orbitalradius #Mystery factor of 2
        print("r0", r0)
        v=np.zeros(2)
        for i in np.arange(2):
            v[i]=np.sqrt(Gconstant*masses[(i+1)%2]/np.abs(r0[i]))
        print(v)
        #r0=orbitalradius #np.sqrt(x0**2+y0**2)
        ux0=-v[0]*sinphi
        uy0=v[1]*cosphi #initial data in y only 
        uz0=np.zeros(2)
    
        a=np.zeros(2)
        a=Fapastron/masses
    
        ax0=-a[0]*cosphi
        ay0=-a[1]*sinphi
        az0=np.zeros(2)

    statevec=[]
    avec=[]
    for i in np.arange(len(x0)):
        stateveci=np.array([x0[i],y0[i],z0[i],ux0[i],uy0[i],uz0[i]])
        aveci=np.array([ax0[i],ay0[i],az0[i]])
        statevec.append(stateveci)
        avec.append(aveci)
    statevecnp=np.array(statevec)
    avecnp=np.array(avec)
    return masses, statevecnp,avecnp

In [16]:
xyuvaeqellipse=getxyuveqtwoellipses(initdateqellipse)
print(xyuvaeqellipse)
masses,xvec,avec=xyuvaeqellipse

initdat 50 [3.14159265 0.        ] 0.5 [-1.  1.] [1.2246468e-16 0.0000000e+00]
coordsep 50
pi
zero
[-50.  50.]
[0. 0.]
0.5
vstuff 75.0 50 0.02666666666666667 0.02 0.003333333333333334 0.013333333333333334
(array([1., 1.]), array([[-7.50000000e+01,  0.00000000e+00,  0.00000000e+00,
        -0.00000000e+00, -5.77350269e-02,  0.00000000e+00],
       [ 7.50000000e+01,  0.00000000e+00,  0.00000000e+00,
        -0.00000000e+00,  5.77350269e-02,  0.00000000e+00]]), array([[ 4.44444444e-05, -0.00000000e+00,  0.00000000e+00],
       [-4.44444444e-05, -0.00000000e+00,  0.00000000e+00]]))


In [17]:
def RK4implicit(h,t,xvec,f): #not a finite difference so no step in y
    k1= h*f(t,xvec)
    k2=h*f(t+h/2, xvec+k1/2)
    k3=h*f(t+h/2,xvec+k2/2)
    k4=h*f(t+h,xvec+k3)
    return t+h, xvec+1/6.*(k1+2.*k2+2.*k3+k4)

In [45]:
class OrbitDiffEq:
    def __init__(self,masses,xvec,avec,t0):
        self.masses=masses
        self.xvec=xvec
        self.ti=t0
        self.avec=avec
    def dxidt(self,t,xvec):
        return xvec[:,3].transpose()
    def dyidt(self,t,xvec):
        return xvec[:,4].transpose()
    def dzidt(self,t,xvec):
        return xvec[:,5].transpose()
    def dvxidt(self,t,xvec):
        #return axi[m]
        axii=np.zeros(len(xvec[:,0]))
        rii=np.sqrt(xvec[:,0]**2+xvec[:,1]**2+xvec[:,2]**2)
        Gconstant=1 #6.408*10**-11
        for k in np.arange(len(rii)):
            for j in np.arange(len(rii)):
                if j!=k:
                    #print(j,k,xvec[j,0],xvec[k,0],xvec[j,0]-xvec[k,0])
                    rreljk=np.abs((xvec[j,0] - xvec[k,0])**2+(xvec[j,1]-xvec[k,1])**2+(xvec[j,2]-xvec[k,2])**2)**(1./2.)
                    axii[j]-=Gconstant*self.masses[k]*(xvec[j,0]  - xvec[k,0])/rreljk**3
        return axii
    def dvyidt(self,t,xvec):
        #return axi[m]
        ayii=np.zeros(len(xvec[:,0]))
        rii=np.sqrt(xvec[:,0]**2+xvec[:,1]**2+xvec[:,2]**2)
        Gconstant=1 #6.408*10**-11
        for k in np.arange(len(rii)):
            for j in np.arange(len(rii)):
                if j!=k:
                    rreljk=np.abs((xvec[j,0] - xvec[k,0])**2+(xvec[j,1]-xvec[k,1])**2+(xvec[j,2]-xvec[k,2])**2)**(1./2.)
                    ayii[j]-=Gconstant*self.masses[k]*(xvec[j,1]  - xvec[k,1])/rreljk**3
        return ayii
    def dvzidt(self,t,xvec):
        #return axi[m]
        azii=np.zeros(len(xvec[:,0]))
        rii=np.sqrt(xvec[:,0]**2+xvec[:,1]**2+xvec[:,2]**2)
        Gconstant=1 #6.408*10**-11
        for k in np.arange(len(rii)):
            for j in np.arange(len(rii)):
                if j!=k:
                    rreljk=np.abs((xvec[j,0] - xvec[k,0])**2+(xvec[j,1]-xvec[k,1])**2+(xvec[j,2]-xvec[k,2])**2)**(1./2.)
                    azii[j]-=Gconstant*self.masses[k]*(xvec[j,2]  - xvec[k,2])/rreljk**3
        return azii
    
    def dvecdt(self,t,xvec):
        avec=np.array([self.dvxidt(self,xvec),self.dvyidt(self,xvec),self.dvzidt(self,xvec)])
        avecT=avec.transpose()
        self.avec=avecT
        dvec2=np.array([self.dxidt(t,xvec),self.dyidt(t,xvec),self.dzidt(self,xvec),avec[0,:],avec[1,:],avec[2,:]])
                       
        dvec2T=dvec2.transpose()
        return dvec2T
    def updateINTERNAL(self,xvecii,tii):
        self.xvec=xvecii
        self.ti=tii
        return self
    def update(self,xvecii,avecii,tii):
        self.xvec=xvecii
        self.avec=avecii,
        self.ti=tii
    def print2D(self):
        print(self.masses,self.xvec,self.ti)
        return self
    def list2D(self):
        return self.masses,self.xvec,self.avec,self.ti
    def timestepRK4ODE(self,step,dt):

    
        h=dt
        #tnew,ynew, intval=RK4(h,t,y,f)
        #m represents choices of mass
        i=step
        
        tnew,intvalxvec=RK4implicit(h,self.ti,self.xvec,self.dvecdt)
        #xii = intvalx
        #tnew,intvalvx=RK4implicit(h,self.ti,self.vxi,self.dvxidt)
        #vxii=intvalvx
        #tnew,intvaly=RK4implicit(h,self.ti,self.yi,self.dyidt)
        #yii = intvaly
        #tnew,intvalvy=RK4implicit(h,self.ti,self.vyi,self.dvyidt)
        #vyii=intvalvy
        #tnew,intvalz=RK4implicit(h,self.ti,self.zi,self.dzidt)
        #zii = intvalz
        #tnew,intvalvz=RK4implicit(h,self.ti,self.vzi,self.dvzidt)
        #vzii=intvalvz
        #axii=self.dvxidt(self.ti,np.zeros(np.shape(self.xi)))
        #ayii=self.dvyidt(self.ti,np.zeros(np.shape(self.yi)))
        #azii=self.dvzidt(self.ti,np.zeros(np.shape(self.zi)))
 
        #print(xii)
        self.updateINTERNAL(intvalxvec,tnew)
        return self.masses, self.xvec,self.avec,self.ti

In [46]:
masses, xvec,avec=xyuvaeqellipse
ODEeq= OrbitDiffEq(masses,xvec,avec,0.0)
ODEeq.print2D()
ODEeq.print2D()

[1. 1.] [[-7.50000000e+01  0.00000000e+00  0.00000000e+00 -0.00000000e+00
  -5.77350269e-02  0.00000000e+00]
 [ 7.50000000e+01  0.00000000e+00  0.00000000e+00 -0.00000000e+00
   5.77350269e-02  0.00000000e+00]] 0.0
[1. 1.] [[-7.50000000e+01  0.00000000e+00  0.00000000e+00 -0.00000000e+00
  -5.77350269e-02  0.00000000e+00]
 [ 7.50000000e+01  0.00000000e+00  0.00000000e+00 -0.00000000e+00
   5.77350269e-02  0.00000000e+00]] 0.0


In [56]:
dt=1 #natural units #*31556926 #seconds per year
numsteps=8000
masses,xvec0,avec0=xyuvaeqellipse
ODEeq= OrbitDiffEq(masses,xvec0,avec0,0.0)
ODEeq.print2D()
t=0.0
star1x=[]
star2x=[]
star1a=[]
star2a=[]
times=[]
for i in np.arange(1,numsteps):
    masses,xvec,avec,t=ODEeq.timestepRK4ODE(i,dt)
    ODEeq.print2D()
    star1x.append(xvec[0])
    star2x.append(xvec[1])
    star1a.append(avec[0])
    star2a.append(avec[1])
    times.append(t)



[1. 1.] [[-7.50000000e+01  0.00000000e+00  0.00000000e+00 -0.00000000e+00
  -5.77350269e-02  0.00000000e+00]
 [ 7.50000000e+01  0.00000000e+00  0.00000000e+00 -0.00000000e+00
   5.77350269e-02  0.00000000e+00]] 0.0
[1. 1.] [[-7.49999778e+01 -5.77350212e-02  0.00000000e+00  4.44444401e-05
  -5.77350098e-02  0.00000000e+00]
 [ 7.49999778e+01  5.77350212e-02  0.00000000e+00 -4.44444401e-05
   5.77350098e-02  0.00000000e+00]] 1.0
[1. 1.] [[-7.49999111e+01 -1.15470008e-01  0.00000000e+00  8.88888538e-05
  -5.77349585e-02  0.00000000e+00]
 [ 7.49999111e+01  1.15470008e-01  0.00000000e+00 -8.88888538e-05
   5.77349585e-02  0.00000000e+00]] 2.0
[1. 1.] [[-7.49998000e+01 -1.73204927e-01  0.00000000e+00  1.33333215e-04
  -5.77348730e-02  0.00000000e+00]
 [ 7.49998000e+01  1.73204927e-01  0.00000000e+00 -1.33333215e-04
   5.77348730e-02  0.00000000e+00]] 3.0
[1. 1.] [[-7.49996444e+01 -2.30939743e-01  0.00000000e+00  1.77777497e-04
  -5.77347532e-02  0.00000000e+00]
 [ 7.49996444e+01  2.30939743e-

   5.69678894e-02  0.00000000e+00]] 212.0
[1. 1.] [[-7.39940568e+01 -1.22425308e+01  0.00000000e+00  9.42430457e-03
  -5.69606515e-02  0.00000000e+00]
 [ 7.39940568e+01  1.22425308e+01  0.00000000e+00 -9.42430457e-03
   5.69606515e-02  0.00000000e+00]] 213.0
[1. 1.] [[-7.39846106e+01 -1.22994878e+01  0.00000000e+00  9.46815010e-03
  -5.69533798e-02  0.00000000e+00]
 [ 7.39846106e+01  1.22994878e+01  0.00000000e+00 -9.46815010e-03
   5.69533798e-02  0.00000000e+00]] 214.0
[1. 1.] [[-7.39751205e+01 -1.23564375e+01  0.00000000e+00  9.51199003e-03
  -5.69460743e-02  0.00000000e+00]
 [ 7.39751205e+01  1.23564375e+01  0.00000000e+00 -9.51199003e-03
   5.69460743e-02  0.00000000e+00]] 215.0
[1. 1.] [[-7.39655866e+01 -1.24133799e+01  0.00000000e+00  9.55582431e-03
  -5.69387351e-02  0.00000000e+00]
 [ 7.39655866e+01  1.24133799e+01  0.00000000e+00 -9.55582431e-03
   5.69387351e-02  0.00000000e+00]] 216.0
[1. 1.] [[-7.39560088e+01 -1.24703150e+01  0.00000000e+00  9.59965294e-03
  -5.69313622e-0

   5.47577092e-02  0.00000000e+00]] 419.0
[1. 1.] [[-7.11140288e+01 -2.38284475e+01  0.00000000e+00  1.83431474e-02
  -5.47436049e-02  0.00000000e+00]
 [ 7.11140288e+01  2.38284475e+01  0.00000000e+00 -1.83431474e-02
   5.47436049e-02  0.00000000e+00]] 420.0
[1. 1.] [[-7.10956646e+01 -2.38831840e+01  0.00000000e+00  1.83852836e-02
  -5.47294681e-02  0.00000000e+00]
 [ 7.10956646e+01  2.38831840e+01  0.00000000e+00 -1.83852836e-02
   5.47294681e-02  0.00000000e+00]] 421.0
[1. 1.] [[-7.10772582e+01 -2.39379064e+01  0.00000000e+00  1.84274089e-02
  -5.47152989e-02  0.00000000e+00]
 [ 7.10772582e+01  2.39379064e+01  0.00000000e+00 -1.84274089e-02
   5.47152989e-02  0.00000000e+00]] 422.0
[1. 1.] [[-7.10588098e+01 -2.39926146e+01  0.00000000e+00  1.84695233e-02
  -5.47010973e-02  0.00000000e+00]
 [ 7.10588098e+01  2.39926146e+01  0.00000000e+00 -1.84695233e-02
   5.47010973e-02  0.00000000e+00]] 423.0
[1. 1.] [[-7.10403192e+01 -2.40473086e+01  0.00000000e+00  1.85116268e-02
  -5.46868632e-0

[1. 1.] [[-6.73203391e+01 -3.30601262e+01  0.00000000e+00  2.54496970e-02
  -5.18232212e-02  0.00000000e+00]
 [ 6.73203391e+01  3.30601262e+01  0.00000000e+00 -2.54496970e-02
   5.18232212e-02  0.00000000e+00]] 593.0
[1. 1.] [[-6.72948695e+01 -3.31119396e+01  0.00000000e+00  2.54895830e-02
  -5.18036147e-02  0.00000000e+00]
 [ 6.72948695e+01  3.31119396e+01  0.00000000e+00 -2.54895830e-02
   5.18036147e-02  0.00000000e+00]] 594.0
[1. 1.] [[-6.72693600e+01 -3.31637334e+01  0.00000000e+00  2.55294539e-02
  -5.17839774e-02  0.00000000e+00]
 [ 6.72693600e+01  3.31637334e+01  0.00000000e+00 -2.55294539e-02
   5.17839774e-02  0.00000000e+00]] 595.0
[1. 1.] [[-6.72438106e+01 -3.32155075e+01  0.00000000e+00  2.55693096e-02
  -5.17643095e-02  0.00000000e+00]
 [ 6.72438106e+01  3.32155075e+01  0.00000000e+00 -2.55693096e-02
   5.17643095e-02  0.00000000e+00]] 596.0
[1. 1.] [[-6.72182213e+01 -3.32672620e+01  0.00000000e+00  2.56091502e-02
  -5.17446109e-02  0.00000000e+00]
 [ 6.72182213e+01  3.32

[1. 1.] [[-6.21734504e+01 -4.19459422e+01  0.00000000e+00  3.22900013e-02
  -4.78611444e-02  0.00000000e+00]
 [ 6.21734504e+01  4.19459422e+01  0.00000000e+00 -3.22900013e-02
   4.78611444e-02  0.00000000e+00]] 771.0
[1. 1.] [[-6.21411420e+01 -4.19937909e+01  0.00000000e+00  3.23268353e-02
  -4.78362734e-02  0.00000000e+00]
 [ 6.21411420e+01  4.19937909e+01  0.00000000e+00 -3.23268353e-02
   4.78362734e-02  0.00000000e+00]] 772.0
[1. 1.] [[-6.21087967e+01 -4.20416147e+01  0.00000000e+00  3.23636501e-02
  -4.78113740e-02  0.00000000e+00]
 [ 6.21087967e+01  4.20416147e+01  0.00000000e+00 -3.23636501e-02
   4.78113740e-02  0.00000000e+00]] 773.0
[1. 1.] [[-6.20764147e+01 -4.20894136e+01  0.00000000e+00  3.24004457e-02
  -4.77864463e-02  0.00000000e+00]
 [ 6.20764147e+01  4.20894136e+01  0.00000000e+00 -3.24004457e-02
   4.77864463e-02  0.00000000e+00]] 774.0
[1. 1.] [[-6.20439959e+01 -4.21371876e+01  0.00000000e+00  3.24372221e-02
  -4.77614903e-02  0.00000000e+00]
 [ 6.20439959e+01  4.21

...

[1. 1.] [[-5.17593537e+01  5.42767842e+01  0.00000000e+00 -4.17822880e-02
  -3.98443691e-02  0.00000000e+00]
 [ 5.17593537e+01 -5.42767842e+01  0.00000000e+00  4.17822880e-02
   3.98443691e-02  0.00000000e+00]] 7111.0
[1. 1.] [[-5.18011206e+01  5.42369238e+01  0.00000000e+00 -4.17516034e-02
  -3.98765213e-02  0.00000000e+00]
 [ 5.18011206e+01 -5.42369238e+01  0.00000000e+00  4.17516034e-02
   3.98765213e-02  0.00000000e+00]] 7112.0
[1. 1.] [[-5.18428569e+01  5.41970312e+01  0.00000000e+00 -4.17208941e-02
  -3.99086498e-02  0.00000000e+00]
 [ 5.18428569e+01 -5.41970312e+01  0.00000000e+00  4.17208941e-02
   3.99086498e-02  0.00000000e+00]] 7113.0
[1. 1.] [[-5.18845624e+01  5.41571065e+01  0.00000000e+00 -4.16901600e-02
  -3.99407548e-02  0.00000000e+00]
 [ 5.18845624e+01 -5.41571065e+01  0.00000000e+00  4.16901600e-02
   3.99407548e-02  0.00000000e+00]] 7114.0
[1. 1.] [[-5.19262372e+01  5.41171497e+01  0.00000000e+00 -4.16594012e-02
  -3.99728360e-02  0.00000000e+00]
 [ 5.19262372e+01 -

   4.48442007e-02  0.00000000e+00]] 7277.0
[1. 1.] [[-5.82906721e+01  4.71931939e+01  0.00000000e+00 -3.63293376e-02
  -4.48721803e-02  0.00000000e+00]
 [ 5.82906721e+01 -4.71931939e+01  0.00000000e+00  3.63293376e-02
   4.48721803e-02  0.00000000e+00]] 7278.0
[1. 1.] [[-5.83269841e+01  4.71483077e+01  0.00000000e+00 -3.62947842e-02
  -4.49001333e-02  0.00000000e+00]
 [ 5.83269841e+01 -4.71483077e+01  0.00000000e+00  3.62947842e-02
   4.49001333e-02  0.00000000e+00]] 7279.0
[1. 1.] [[-5.83632616e+01  4.71033936e+01  0.00000000e+00 -3.62602093e-02
  -4.49280598e-02  0.00000000e+00]
 [ 5.83632616e+01 -4.71033936e+01  0.00000000e+00  3.62602093e-02
   4.49280598e-02  0.00000000e+00]] 7280.0
[1. 1.] [[-5.83995045e+01  4.70584516e+01  0.00000000e+00 -3.62256129e-02
  -4.49559596e-02  0.00000000e+00]
 [ 5.83995045e+01 -4.70584516e+01  0.00000000e+00  3.62256129e-02
   4.49559596e-02  0.00000000e+00]] 7281.0
[1. 1.] [[-5.84357128e+01  4.70134817e+01  0.00000000e+00 -3.61909951e-02
  -4.498383

   4.89012806e-02  0.00000000e+00]] 7434.0
[1. 1.] [[-6.35553001e+01  3.98211480e+01  0.00000000e+00 -3.06543340e-02
  -4.89248929e-02  0.00000000e+00]
 [ 6.35553001e+01 -3.98211480e+01  0.00000000e+00  3.06543340e-02
   4.89248929e-02  0.00000000e+00]] 7435.0
[1. 1.] [[-6.35859356e+01  3.97722113e+01  0.00000000e+00 -3.06166625e-02
  -4.89484761e-02  0.00000000e+00]
 [ 6.35859356e+01 -3.97722113e+01  0.00000000e+00  3.06166625e-02
   4.89484761e-02  0.00000000e+00]] 7436.0
[1. 1.] [[-6.36165335e+01  3.97232510e+01  0.00000000e+00 -3.05789729e-02
  -4.89720303e-02  0.00000000e+00]
 [ 6.36165335e+01 -3.97232510e+01  0.00000000e+00  3.05789729e-02
   4.89720303e-02  0.00000000e+00]] 7437.0
[1. 1.] [[-6.36470936e+01  3.96742672e+01  0.00000000e+00 -3.05412651e-02
  -4.89955555e-02  0.00000000e+00]
 [ 6.36470936e+01 -3.96742672e+01  0.00000000e+00  3.05412651e-02
   4.89955555e-02  0.00000000e+00]] 7438.0
[1. 1.] [[-6.36776160e+01  3.96252599e+01  0.00000000e+00 -3.05035393e-02
  -4.901905

[1. 1.] [[-6.94016653e+01  2.84325316e+01  0.00000000e+00 -2.18873730e-02
  -5.34254269e-02  0.00000000e+00]
 [ 6.94016653e+01 -2.84325316e+01  0.00000000e+00  2.18873730e-02
   5.34254269e-02  0.00000000e+00]] 7657.0
[1. 1.] [[-6.94235321e+01  2.83790978e+01  0.00000000e+00 -2.18462396e-02
  -5.34422599e-02  0.00000000e+00]
 [ 6.94235321e+01 -2.83790978e+01  0.00000000e+00  2.18462396e-02
   5.34422599e-02  0.00000000e+00]] 7658.0
[1. 1.] [[-6.94453578e+01  2.83256471e+01  0.00000000e+00 -2.18050933e-02
  -5.34590613e-02  0.00000000e+00]
 [ 6.94453578e+01 -2.83256471e+01  0.00000000e+00  2.18050933e-02
   5.34590613e-02  0.00000000e+00]] 7659.0
[1. 1.] [[-6.94671423e+01  2.82721796e+01  0.00000000e+00 -2.17639340e-02
  -5.34758311e-02  0.00000000e+00]
 [ 6.94671423e+01 -2.82721796e+01  0.00000000e+00  2.17639340e-02
   5.34758311e-02  0.00000000e+00]] 7660.0
[1. 1.] [[-6.94888856e+01  2.82186954e+01  0.00000000e+00 -2.17227619e-02
  -5.34925691e-02  0.00000000e+00]
 [ 6.94888856e+01 -

[1. 1.] [[-7.27905475e+01  1.80703124e+01  0.00000000e+00 -1.39105330e-02
  -5.60341896e-02  0.00000000e+00]
 [ 7.27905475e+01 -1.80703124e+01  0.00000000e+00  1.39105330e-02
   5.60341896e-02  0.00000000e+00]] 7846.0
[1. 1.] [[-7.28044365e+01  1.80142729e+01  0.00000000e+00 -1.38673937e-02
  -5.60448813e-02  0.00000000e+00]
 [ 7.28044365e+01 -1.80142729e+01  0.00000000e+00  1.38673937e-02
   5.60448813e-02  0.00000000e+00]] 7847.0
[1. 1.] [[-7.28182823e+01  1.79582227e+01  0.00000000e+00 -1.38242463e-02
  -5.60555399e-02  0.00000000e+00]
 [ 7.28182823e+01 -1.79582227e+01  0.00000000e+00  1.38242463e-02
   5.60555399e-02  0.00000000e+00]] 7848.0
[1. 1.] [[-7.28320850e+01  1.79021618e+01  0.00000000e+00 -1.37810906e-02
  -5.60661652e-02  0.00000000e+00]
 [ 7.28320850e+01 -1.79021618e+01  0.00000000e+00  1.37810906e-02
   5.60661652e-02  0.00000000e+00]] 7849.0
[1. 1.] [[-7.28458445e+01  1.78460904e+01  0.00000000e+00 -1.37379268e-02
  -5.60767572e-02  0.00000000e+00]
 [ 7.28458445e+01 -

In [57]:
npstar2x=np.array(star2x)
npstar1x=np.array(star1x)
npstar2a=np.array(star2a)
npstar1a=np.array(star1a)
nptimes=np.array(times)

In [58]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook()
# create a new plot with default tools, using figure
p = figure(title="Elliptical orbit, RK4", plot_width=400, plot_height=400)
t = np.linspace(0, 2*math.pi, 100)
focusx=getfocus(rad0,ecc0)
b=rad0*np.sqrt(1-ecc0**2)
print(b)
print(focusx)

# add a circle renderer with x and y coordinates, size, color, and alpha
p.line(npstar1x[:,0],npstar1x[:,1], line_color="purple", legend="Star 1")
p.line(npstar2x[:,0],npstar2x[:,1], line_color="blue", legend="Star 2")
p.line(focus*np.cos(theta0)+rad0*np.cos(t)*np.cos(theta0)-b*np.sin(t)*np.sin(theta0),focus*np.sin(theta0)+rad0*np.cos(theta0)*np.sin(t)+b*np.sin(theta0)*np.cos(t), line_color="cyan", legend="Ideal ellipse 1")
p.line(-focus*np.cos(theta0)+rad0*np.cos(t)*np.cos(theta0)-b*np.sin(t)*np.sin(theta0),-focus*np.sin(theta0)+rad0*np.cos(theta0)*np.sin(t)+b*np.sin(theta0)*np.cos(t), line_color="black", legend="Ideal ellipse 2")

print(theta0)



p.legend.location = "bottom_left"
p.legend.click_policy="hide"

show(p) # show the

Loading BokehJS ...

43.30127018922193
25.0
0.0
